In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import deepcopy

In [ ]:
from hsr4hci.utils.config import load_config
from hsr4hci.utils.data import load_data
from hsr4hci.utils.fits import save_fits
from hsr4hci.models.hsr import HalfSiblingRegression

## Run a test experiment for one pixel position

In [ ]:
collection_position = (30, 20)
experiment_dir = "../experiments/debug_ortho_test/"
config = load_config(os.path.join(experiment_dir, 'config.json'))

In [ ]:
# Load frames and parallactic angles from HDF file
stack, parang, psf_template = load_data(dataset_config=config['dataset'])

stack -= np.median(stack, axis=0)

In [ ]:
# Instantiate model
hsr = HalfSiblingRegression(config=config)

In [ ]:
hsr.train_position(position=collection_position,
                   stack=stack,
                   parang=parang,
                   psf_template=psf_template)

## Get the collection

In [ ]:
test_collection = hsr.m__collections[collection_position]
plt.imshow(test_collection.m__collection_region_mask)

## Visualize the sources and targets

In [ ]:
planet_position = test_collection.m__planet_positions[200]
planet_position = (int(planet_position[0]), int(planet_position[1]))
planet_position

In [ ]:
pred_position = planet_position #list(test_collection.m__predictors.keys())[30]

In [ ]:
predictor = test_collection.m__predictors[pred_position]
test_sources = test_collection.m__sources[pred_position]
dummy = bool(test_collection.m__use_forward_model * test_collection.m__add_planet_column)
print(test_sources.shape)

In [ ]:
plt.plot(test_sources[:, 500])
plt.title("Example noise source with orthogonalization")

In [ ]:
noise_prediction = predictor.get_noise_prediction(sources=test_sources,
                                                  add_dummy_column=dummy)
plt.plot(noise_prediction, label="prediction")
plt.plot(stack[:, pred_position[0], pred_position[1]], label="original target sequence")
plt.axvline(200, color="r",label="  ~Planet position")
plt.legend()
plt.title("Prediction vs. target sequence \n with orthogonalization")

In [ ]:
residual = stack[:, pred_position[0], pred_position[1]] - noise_prediction
residual -= np.median(residual)
plt.plot(residual, label="Residual")
plt.axvline(200, color="r",label=" ~Planet position")
plt.legend()
plt.title("Residual with orthogonalization")

## Run the same experiment with

In [ ]:
config_no_ortho = deepcopy(config)
config_no_ortho["experiment"]['sources']["preprocessing"] = [{'type': 'standardize', 'parameters': {}},]

In [ ]:
# Instantiate model
hsr_no_ortho = HalfSiblingRegression(config=config_no_ortho)

In [ ]:
hsr_no_ortho.train_position(position=collection_position,
                            stack=stack,
                            parang=parang,
                            psf_template=psf_template)

In [ ]:
test_collection_no_ortho = hsr_no_ortho.m__collections[collection_position]
plt.imshow(test_collection_no_ortho.m__collection_region_mask)

## Visualize the sources and targets no ortho

In [ ]:
predictor_no_ortho = test_collection_no_ortho.m__predictors[pred_position]
test_sources_no_ortho = test_collection_no_ortho.m__sources[pred_position]

dummy_no_ortho = bool(test_collection_no_ortho.m__use_forward_model * test_collection_no_ortho.m__add_planet_column)

In [ ]:
idx=500
plt.plot(test_sources_no_ortho[:, idx])
plt.plot(test_sources[:, idx])

In [ ]:
noise_prediction_no_ortho = predictor_no_ortho.get_noise_prediction(sources=test_sources_no_ortho,
                                                                    add_dummy_column=dummy_no_ortho)

plt.plot(noise_prediction_no_ortho, label="prediction")
plt.plot(stack[:, pred_position[0], pred_position[1]], label="original target sequence")
plt.axvline(200, color="r",label="  ~Planet position")
plt.legend()
plt.title("Prediction vs. target sequence \n without orthogonalization")

In [ ]:
residual_no_ortho = stack[:, pred_position[0], pred_position[1]] - noise_prediction_no_ortho
residual_no_ortho -= np.median(residual_no_ortho)
plt.plot(residual_no_ortho, label="Residual")
plt.axvline(200, color="r",label=" ~Planet position")
plt.legend()
plt.title("Residual without orthogonalization")

## Residual vs original data

In [ ]:
residual = stack[:, pred_position[0], pred_position[1]] - noise_prediction
residual -= np.median(residual)
plt.plot(stack[:, pred_position[0], pred_position[1]], label="original target sequence")
plt.plot(residual, label="Residual")
plt.axvline(200, color="r",label=" ~Planet position")
plt.legend()
plt.title("Residual with orthogonalization")

## Observation: Around the position of the planet from the forward model the residual signal with orthogonalization is very similar to the raw sequence. As we only use the information around this position we might end up with a sequence identical to the input!